In [1]:
!pip install foolbox

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 62 kB 840 kB/s 
     |████████████████████████████████| 170 kB 45.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

In [3]:
from __future__ import print_function
import torch
import numpy as np
import warnings
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
import torchvision.models as models

In [4]:
class Model_Drop(nn.Module):

    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x


In [5]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [6]:
def unc_defense(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model(norm((image+delta).clamp(0,1)))
            output = softmax(output)

            dropout_predictions[i] = output

        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)

        model.eval()
        o = model(norm((image + delta).clamp(0, 1)))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model(norm((image + delta).clamp(0, 1)))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [7]:
warnings.filterwarnings("ignore")


torch.manual_seed(2)
np.random.seed(2)

batch_size = 64
eps = 2./255
alpha = 0.2 * eps
num_iter = 10
eps_l2 = 53./255

eps_little = 0.25/255
alpha_reverse = 0.2 * eps_little
num_iter_reverse = 10

count_successful_reverse = 0
count_unsuccessful_reverse = 0

count_successfull_attack = 0
count_unsuccessfull_attack = 0

transform = transforms.ToTensor()

test_data = datasets.CIFAR10(root='/CIFARDATA', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /CIFARDATA/cifar-10-python.tar.gz to /CIFARDATA


In [8]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_cnn = Model_Drop()
model_cnn.load_state_dict(torch.load("model_cnn_cifar10.pt",map_location=device))
model_cnn.eval()
model_cnn.to(device)

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)

2021-09-09 12:57:26


In [9]:
for i, (image, label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(image))
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]

    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1), preprocessing=preprocessing)
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    attack = L2CarliniWagnerAttack(steps=900)
    fmodel = PyTorchModel(model_cnn, bounds=(0, 1), preprocessing=preprocessing)
    raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    pert = torch.tensor(clipped_advs[0])

    #attack = L2DeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1), preprocessing=preprocessing)
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    #pert = torch.tensor(clipped_advs[0])

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(pert))
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]
    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]

    if inds_wrong_after_attack.shape[0] == 0:
        continue

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]
    pred_pert = pred_pert[inds_wrong_after_attack]

    reversed_pert = unc_defense(pert, model_cnn, eps_little, num_iter_reverse, alpha_reverse)

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(reversed_pert))
        o = softmax(o)

    pred_reverse = o.data.max(1, keepdim=True)[1]
    pred_reverse = pred_reverse.view_as(label)

    inds_correct_after_reverse = np.where(pred_reverse.cpu() == label.cpu())[0]
    inds_wrong_after_reverse = np.where(pred_reverse.cpu() != label.cpu())[0]

    reversed_pert = reversed_pert[inds_correct_after_reverse]
    image = image[inds_correct_after_reverse]
    label = label[inds_correct_after_reverse]
    pert = pert[inds_correct_after_reverse]
    pred_pert = pred_pert[inds_correct_after_reverse]

    inds_correct_after_reverse = inds_correct_after_reverse.tolist()
    inds_wrong_after_reverse = inds_wrong_after_reverse.tolist()

    count_successful_reverse += len(inds_correct_after_reverse)
    count_unsuccessful_reverse += len(inds_wrong_after_reverse)

    if i%10 == 0:
        print(i)



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [10]:
print("Number of successful reverse operation is : ", count_successful_reverse)
print("Number of unsuccessful reverse operation is : ", count_unsuccessful_reverse)

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful reverse operation is :  4904
Number of unsuccessful reverse operation is :  102
2021-09-09 17:03:58
